## Общее описание:

Необходимо визуализировать данные (в PowerBi) о портфеле брендов табачных и никотинсодержащих изделий на российском рынке, продаваемых в 2021 году, показав:
- продажи табачных и никотинсодержащих изделий в разрезе типа продукции;
- долю каждого из типа упаковки, в которых осуществляется оборот продукции;
- объемы, в которых продаются жидкости для ЭСДН;
- долю каждого из видов упаковки, в которых продаются жидкости.

## Описание данных

Предоставлены данные о портфеле брендов табачных и никотинсодержащих изделий на российском рынке за 2021 год. 

**Описание полей**:

- `Наименование` — наименование бренда
<br><br>
- `GTIN` — глобальный номер товарной продукции в единой международной базе товаров GS1
<br><br>
- `Тип упаковки` — тип упаковки продукции



### План

1. Подготовка к работе. Импорт необходимых библиотек и загрузка данных о брендах
<br><br>
2. Создание столбца с типом продукции на основе столбца с названием бренда
<br><br>
3. На основе данных о брендах создать новые данные, содержащие информацию исключительно о жидкостях для ЭСДН. Создать столбец с информацией об объеме, в которых продаются жидкости на рынке, и столбец о типе упаковки, в которых продаются жидкости.
<br><br>
4. Сохранение данных и дальнейшая работа с ними в PowerBi

<br><br>
</b> 

<hr style='border: 1px solid'>

## 1. Импорт библиотек и загрузка данных 

In [1]:
import pandas as pd

In [2]:
# Читаем данные

df = pd.read_excel('./data/Справочник_Табак_2021.xlsx')

In [3]:
df.head()

,Наименование,GTIN,Тип упаковки,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,MARLBORO DOUBLE MIX,4870010009968,Групповая,NaN,NaN,NaN,NaN
1,KENT BLUE,4600266010599,Групповая,NaN,NaN,NaN,NaN
2,BOND STREET COMPACT BLUE,48743389,Потребительская,NaN,NaN,NaN,NaN
3,ESSE EXCHANGE M,4640030090532,Групповая,NaN,NaN,NaN,NaN
4,"2 картриджа для электронных сигарет RELX, Gard...",6972890396768,Потребительская,NaN,NaN,NaN,NaN


In [4]:
# Смотрим размер данных

df.shape

(2160, 7)

In [5]:
# Проверяем данные на наличие строк-дубликатов

df.loc[df.duplicated()]

,Наименование,GTIN,Тип упаковки,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6


<hr style='border: 1px solid'>

## 2. Преобразование и сохранение данных о брендах 

In [6]:
# Для начала удаляем ненужные столбцы из данных, оставляя только наименование продукции и тип упаковки

df = df.reindex(columns=['Наименование','Тип упаковки'])

In [7]:
df.head()

,Наименование,Тип упаковки
0,MARLBORO DOUBLE MIX,Групповая
1,KENT BLUE,Групповая
2,BOND STREET COMPACT BLUE,Потребительская
3,ESSE EXCHANGE M,Групповая
4,"2 картриджа для электронных сигарет RELX, Gard...",Потребительская


In [8]:
# Переименовываем столбцы для удобства

df.rename(columns={'Наименование':'brand_name','Тип упаковки':'package_type'},inplace=True)

In [9]:
df.head()

,brand_name,package_type
0,MARLBORO DOUBLE MIX,Групповая
1,KENT BLUE,Групповая
2,BOND STREET COMPACT BLUE,Потребительская
3,ESSE EXCHANGE M,Групповая
4,"2 картриджа для электронных сигарет RELX, Gard...",Потребительская


In [10]:
df.loc[df['package_type'] == 2402209000]

,brand_name,package_type
291,Davidoff Reach Purple,2402209000


In [11]:
df.drop(df[df['package_type'] == 2402209000].index, inplace=True)

In [12]:
df

,brand_name,package_type
0,MARLBORO DOUBLE MIX,Групповая
1,KENT BLUE,Групповая
2,BOND STREET COMPACT BLUE,Потребительская
3,ESSE EXCHANGE M,Групповая
4,"2 картриджа для электронных сигарет RELX, Gard...",Потребительская
...,...,...
2155,Жидкость для электронных сигарет Nasty Salt CU...,Потребительская
2156,Жидкость для электронных сигарет Nasty Salt Hi...,Потребительская
2157,Жидкость для электронных сигарет Nasty Salt CU...,Потребительская
2158,Сигареты Davidoff Evolve Purple SSL с капсулой...,Потребительская


In [13]:
# На основе столбца "Наименование" необходимо создать столбец с типом продукции.
# Известно, что продукция типа "сигареты" в столбце "brand_name" как может содержать слово "сигареты", так и 
# содержать просто наименование бренда на английском.
# Будем также относить "картриджи" и жидкости к одному типу продукции - жидкоси для ЭСДН.

# Пишем функцию 

def creating_product_type(value):
    if 'сигареты' in value or 'Сигареты' in value:
        return 'Сигареты'
    elif 'сигарилл' in value or 'Сигарилл' in value:
        return 'Сигариллы'
    elif 'Сигары' in value or 'сигары' in value:
        return 'Сигары'
    elif 'картридж' in value or 'жидкост' in value or 'Жидкост' in value:
        return 'Жидкости для ЭСДН'
    elif 'кальяна' in value:
        return 'Табак для кальяна'
    elif 'бестабачная' in value or 'Бестабачная' in value:
        return 'Бестабачные смеси для кальяна'
    elif 'нагрев' in value or 'Нагрев' in value:
        return 'Нагреваемый табак'
    else:
        return 'Сигареты'

In [14]:
df['product_type'] = df['brand_name'].apply(creating_product_type)

In [15]:
df

,brand_name,package_type,product_type
0,MARLBORO DOUBLE MIX,Групповая,Сигареты
1,KENT BLUE,Групповая,Сигареты
2,BOND STREET COMPACT BLUE,Потребительская,Сигареты
3,ESSE EXCHANGE M,Групповая,Сигареты
4,"2 картриджа для электронных сигарет RELX, Gard...",Потребительская,Жидкости для ЭСДН
...,...,...,...
2155,Жидкость для электронных сигарет Nasty Salt CU...,Потребительская,Жидкости для ЭСДН
2156,Жидкость для электронных сигарет Nasty Salt Hi...,Потребительская,Жидкости для ЭСДН
2157,Жидкость для электронных сигарет Nasty Salt CU...,Потребительская,Жидкости для ЭСДН
2158,Сигареты Davidoff Evolve Purple SSL с капсулой...,Потребительская,Сигареты


In [16]:
# Сохраняем данные о брендах в разрезе типов продукции

df.to_excel('tobacco_mark_2021.xlsx')

<hr style='border: 1px solid'>

## 3. Создаем новые данные, содержащие информацию о жидкостях для ЭСДН

### 3.1. Создаем столбец с данными об объемах продаваемой жидкости

In [17]:
# Фильтруем данные о брендах и оставляем только данные с брендами жидкостей для ЭСДН

liquids = df.query('product_type == "Жидкости для ЭСДН"').copy()

In [18]:
liquids

,brand_name,package_type,product_type
4,"2 картриджа для электронных сигарет RELX, Gard...",Потребительская,Жидкости для ЭСДН
23,Жидкость для электронных сигарет HUNGRY Grape ...,Потребительская,Жидкости для ЭСДН
52,"2 картриджа для электронных сигарет RELX, Gold...",Потребительская,Жидкости для ЭСДН
91,Жидкость для электронных сигарет HUNGRY Tropic...,Потребительская,Жидкости для ЭСДН
108,Жидкость для электронных сигарет The Scandalis...,Потребительская,Жидкости для ЭСДН
...,...,...,...
2153,Жидкость для электронных сигарет Jam Monster L...,Потребительская,Жидкости для ЭСДН
2154,Жидкость для электронных сигарет Custard Monst...,Потребительская,Жидкости для ЭСДН
2155,Жидкость для электронных сигарет Nasty Salt CU...,Потребительская,Жидкости для ЭСДН
2156,Жидкость для электронных сигарет Nasty Salt Hi...,Потребительская,Жидкости для ЭСДН


In [19]:
# Удаляем ненужные столбцы

liquids = liquids.reindex(columns=['brand_name'])

In [20]:
# Необходимо создать новый столбец с данными об объемах в мл продаваемых на рынке жидкостей
# Известно, что данные об объеме продаваемой жидкости содержатся в конце наиенования бренда

liquids['volume'] = liquids['brand_name'].apply(lambda x: x.split(' ')[-2:])

In [21]:
liquids

,brand_name,volume
4,"2 картриджа для электронных сигарет RELX, Gard...","[3.8, ml]"
23,Жидкость для электронных сигарет HUNGRY Grape ...,"[120мл, 3мг]"
52,"2 картриджа для электронных сигарет RELX, Gold...","[3.8, ml]"
91,Жидкость для электронных сигарет HUNGRY Tropic...,"[120мл, 3мг]"
108,Жидкость для электронных сигарет The Scandalis...,"[58мл, 0мг]"
...,...,...
2153,Жидкость для электронных сигарет Jam Monster L...,"[100мл, 3мг]"
2154,Жидкость для электронных сигарет Custard Monst...,"[30мл, 20мг]"
2155,Жидкость для электронных сигарет Nasty Salt CU...,"[30мл, 20мг]"
2156,Жидкость для электронных сигарет Nasty Salt Hi...,"[30мл, 20мг]"


In [22]:
# Данные об объеме записаны по-разному, в зависимости от того, картридж это или флакон
# Необходимо оставить только числовое значение продаваемых объемов

def extracting_volume(v):
#     v = v.replace('[','').replace(']','')
    if 'ml' in v:
        return v[0] + 'мл'
    if 'мл' in v[0]:
        return v[0]
    else:
        return 'no info'
        

In [23]:
liquids['volume'] = liquids.volume.apply(extracting_volume)

In [24]:
liquids

,brand_name,volume
4,"2 картриджа для электронных сигарет RELX, Gard...",3.8мл
23,Жидкость для электронных сигарет HUNGRY Grape ...,120мл
52,"2 картриджа для электронных сигарет RELX, Gold...",3.8мл
91,Жидкость для электронных сигарет HUNGRY Tropic...,120мл
108,Жидкость для электронных сигарет The Scandalis...,58мл
...,...,...
2153,Жидкость для электронных сигарет Jam Monster L...,100мл
2154,Жидкость для электронных сигарет Custard Monst...,30мл
2155,Жидкость для электронных сигарет Nasty Salt CU...,30мл
2156,Жидкость для электронных сигарет Nasty Salt Hi...,30мл


In [25]:
liquids.volume.unique()

array(['3.8мл', '120мл', '58мл', '30мл', '100мл', 'no info', '1.9мл',
       '60мл', '80мл', '50мл', 'мл', 'мл,'], dtype=object)

In [26]:
# Убираем из данных строки с неизвестными данными по объему жидкостей

liquids = liquids.query('(volume != "мл") & (volume != "мл,") &  (volume != "no info")')

In [27]:
liquids.volume.unique()

array(['3.8мл', '120мл', '58мл', '30мл', '100мл', '1.9мл', '60мл', '80мл',
       '50мл'], dtype=object)

<hr>

### 3.2. Создаем столбец с данными об упаковках, в которых продаются жидкости

In [30]:
# Известно, что жидкости продаются либо в картриджах, либо во флаконах.
# С помощью функции будем вычислять значение типа упаковки

def type_package(value):
    if 'картридж' in value or 'Картридж' in value:
        return 'Картриджи'
    else:
        return 'Флаконы'

In [31]:
liquids['type_package'] = liquids['brand_name'].apply(type_package)

C:\Users\Kout2\AppData\Local\Temp\ipykernel_16248\1648065515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  liquids['type_package'] = liquids['brand_name'].apply(type_package)


In [34]:
liquids.head()

,brand_name,volume,type_package
4,"2 картриджа для электронных сигарет RELX, Gard...",3.8мл,Картриджи
23,Жидкость для электронных сигарет HUNGRY Grape ...,120мл,Флаконы
52,"2 картриджа для электронных сигарет RELX, Gold...",3.8мл,Картриджи
91,Жидкость для электронных сигарет HUNGRY Tropic...,120мл,Флаконы
108,Жидкость для электронных сигарет The Scandalis...,58мл,Флаконы


In [33]:
# Сохраняем данные

liquids.to_excel('liquids_info.xlsx')